<a href="https://colab.research.google.com/github/SezerBugday/photogrammetry_meshroom_colab_flask/blob/main/Bitirme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 main.py

In [35]:
from pyngrok import ngrok

import os  
import shutil  


from flask import Flask, flash, request, redirect, url_for,render_template
from werkzeug.utils import secure_filename

port_no = 5000

import sqlite3 as sql


con = sql.connect("/content/photogrammetry_meshroom_colab_flask/sezerdb.db", check_same_thread=False)
cursor = con.cursor()

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import sys

import random


In [34]:
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

TEMPLATE = "/content/photogrammetry_meshroom_colab_flask/templates"
app = Flask(__name__, template_folder=TEMPLATE)
app._static_folder = '/content/static/'

#bootsrapt = Bootstrap(app)
islem_durumu=False;


app.config['UPLOAD_FOLDER'] = "/content/kirli"

ngrok.set_auth_token("2DJyoS7QMsne6ZsjPBWlik5uBBv_37hajTNHz6NFWXeczXf7A")
public_url = ngrok.connect(port_no).public_url

print(f"To acces the Gloable link please click {public_url}")


@app.route("/")
def main_page():

    print("Anasayfaya donuldu")
    return render_template("HomePage.html")

@app.route("/uploadimages", methods=["GET", "POST"])
def upload_file():
    print("resimler secildi")
    if request.method == 'POST':
        files = request.files.getlist("image")
        for file in files:
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], file.filename))
            print("Görseller kaydedildi")
            return redirect(url_for('Show_Process'))
    return render_template("ShowSelectedImages.html")
  
@app.route("/process")
def Show_Process():
  if(islem_durumu==False):
    return redirect(url_for('Process'))
  return render_template("processing.html")
  print("isleniyor")

  
@app.route("/sezer")
def Process():
  return render_template("showModel.html")
  return "çalisti"
  os.system("rembg p /content/kirli   /content/temiz")
  os.system("./meshroom/Meshroom-2019.1.0/meshroom_photogrammetry --input /content/temiz --output /content/modelFolder")
  os.system("obj2gltf -i /content/modelFolder/model.obj -o /content/static/model.glb")
  
@app.route("/SingUp", methods=["GET", "POST"])
def SingUp():
   if request.method == 'POST':
        name = request.form.get("name")
        surname = request.form.get("surname")
        global email
        email = request.form.get("email")
        password = request.form.get("password")
        print("before db")
        print(name)
        #cursor.execute("INSERT INTO Users VALUES(name,surname,password,email)")
        cursor.execute("insert into Users values(?,?,?,?,?,?)",(name, surname, password,email,0,None))
        con.commit()
        print("after db")
        # Gmail email sunucusuna bağlanıyoruz
        try:
            global ver_code
            ver_code = random.randint(100000,999999)
            print("vercode", ver_code)
            mail = smtplib.SMTP("smtp.gmail.com",587)
            mail.ehlo()
            mail.starttls()
            mail.login("sezerbugday@gmail.com", "vqrwxdipzjoftucq")
            mesaj = MIMEMultipart()
            mesaj["From"] = "3D AR Object Platform "           # Gönderen
            mesaj["Subject"] = "Verification Code"   # Konusu
            mesaj["To"] = email
            body = "Your Verification Code :  "+ str(ver_code) 
            body_text = MIMEText(body, "plain")  #
            mesaj.attach(body_text)
            mail.sendmail(mesaj["From"], mesaj["To"], mesaj.as_string())
            print("Mail başarılı bir şekilde gönderildi.")
            mail.close()
            return render_template("SentVerificationCode.html")
        # Eğer mesaj gönderirken hata olursa, hata mesajını konsole yazdırıyorum.
        except:
            print("Hata:", sys.exc_info()[0])
            
   return render_template("SingUp.html")

@app.route("/SentCode", methods=["GET", "POST"])
def SentCode():
  if request.method == 'POST':
    code = request.form.get("sifre")
    print("gelen kod", ver_code)
    print("girilen kod", code)

    if(int(code) == ver_code):
      cursor.execute("UPDATE Users SET Is_Active = 1 WHERE Email = email" )
      con.commit()
      return render_template("Login.html",yazi="Tebrikler mailiniz onaylandı şimdi giriş yapabilirsiniz.")
    else:
      return render_template("SentVerificationCode.html",yazi="Mail onaylanmadı kodu tekrar giriniz.")
  return render_template("SentVerificationCode.html")

       
if __name__ == "__main__":
    app.run()

To acces the Gloable link please click http://42fe-35-232-247-204.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [23/Dec/2022 17:24:16] "GET / HTTP/1.1" 200 -


Anasayfaya donuldu


INFO:werkzeug:127.0.0.1 - - [23/Dec/2022 17:24:16] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [23/Dec/2022 17:24:25] "GET /uploadimages HTTP/1.1" 200 -


resimler secildi
resimler secildi


...

gelen kod 469635
girilen kod 34434343


INFO:werkzeug:127.0.0.1 - - [23/Dec/2022 17:25:19] "POST /SentCode HTTP/1.1" 200 -


gelen kod 469635
girilen kod 434365


INFO:werkzeug:127.0.0.1 - - [23/Dec/2022 17:25:24] "POST /SentCode HTTP/1.1" 200 -


gelen kod 469635
girilen kod 469635


In [23]:
print("vercode", ver_code)

vercode 934168


In [ ]:


# Gmail email sunucusuna bağlanıyoruz
try:
    mail = smtplib.SMTP("smtp.gmail.com",587)
    mail.ehlo()
    mail.starttls()
    mail.login("sezerbugday@gmail.com", "vqrwxdipzjoftucq")

    mesaj = MIMEMultipart()
    mesaj["From"] = "3D AR Object Platform "           # Gönderen
    mesaj["Subject"] = "Verification Code"    # Konusu
    mesaj["To"] = "sezerbugday2@gmail.com "
    body = " Bu mesaj sezer tarafından python ile gönderilmiştir "

    body_text = MIMEText(body, "plain")  #
    mesaj.attach(body_text)

    mail.sendmail(mesaj["From"], mesaj["To"], mesaj.as_string())
    print("Mail başarılı bir şekilde gönderildi.")
    mail.close()

# Eğer mesaj gönderirken hata olursa, hata mesajını konsole yazdırıyorum.
except:
    print("Hata:", sys.exc_info()[0])

Mail başarılı bir şekilde gönderildi.


In [ ]:
import sqlite3 as sql

con = sql.connect("/content/sezerdb.db", check_same_thread=False)
cursor = con.cursor()

In [ ]:
name="1"
surname="2"
email="3"
password ="4"
#cursor.execute("insert into Users values('Name','Surname','Password','Email','Is_Active')",(name, surname, email,password,5))
cursor.execute("insert into Users values(?,?,?,?,?,?)",(name, surname, email,password,0,None))
con.commit()

In [37]:
cursor.execute("Select * From Users")
veri = cursor.fetchall() 
print(veri)

[('sezer', 'bugday', '123', 'deneme', 1, 1), (None, None, None, None, None, 2), ('1', '2', '3', '4', 1, 6), ('Name', 'Surname', 'Password', 'Email', 1, 7), ('9', '9', '9', '9', 1, 8), ('Name', 'Surname', 'Password', 'Email', 1, 9), ('9', '9', '9', '9', 1, 10), ('Name', 'Surname', 'Password', 'Email', 1, 11), ('9', '9', '9', '9', 1, 12), ('nisa nur', 'dalkıran', '220322', 'nnurdalkiran@gmail.com', 1, 13), ('sfd', 'sdffsd', 'dsffdssfd', 'sezerbugday@gmail2.com', 1, 14), ('sezer', 'bugday', '220322', 'sezerbugday2@gmail.com', 1, 15), ('sezer', 'bugday', '220322', 'sezerbugday2@gmail.com', 1, 16), ('fds', 'sdf', '15155', 'sezerbugday2@gmail.com', 1, 17), ('fsd', 'sdf', 'wfsddfs', 'sezerbugday2@gmail.com', 1, 18), ('saddsa', 'asasdas', 'sfasfd', 'sezerbugday2@gmail.com', 1, 19), ('sd', 'sd', 'sd', 'sezerbugday2@gmail.com', 1, 20), ('dsgd', 'sgsdf', 'sfdsfd', 'sezerbugday2@gmail.com', 1, 21), ('sfdsdf', 'sdfs', 'sfsfd', 'sezerbugday2@gmail.com', 1, 22), ('sff', 'sfssfs', 'fsdsfd', 'sezerbugd

In [ ]:
def VeriSil():
    cursor.execute("DELETE FROM Users")
    con.commit()
VeriSil()

In [46]:
!git status


On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git reset HEAD <file>..." to unstage)

	modified:   sezerdb.db
	new file:   templates/Login.html
	modified:   templates/SentVerificationCode.html



In [41]:
!git add --all  

In [45]:
!git config --global user.email "sezerbugday2@gmail.com"

In [47]:
!git commit -a -m "dogrulama kodu yapıldı"


[main 76178e0] dogrulama kodu yapıldı
 3 files changed, 31 insertions(+), 6 deletions(-)
 create mode 100644 templates/Login.html


In [49]:
!git branch

* main


In [50]:
!git push origin main

Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 1.89 KiB | 1.89 MiB/s, done.
Total 6 (delta 4), reused 0 (delta 0)
remote: Resolving deltas: 100% (4/4), completed with 4 local objects.
To https://github.com/SezerBugday/photogrammetry_meshroom_colab_flask.git
   57018c8..76178e0  main -> main
